Libraries and Display options

In [8]:
import pandas as pd
import numpy as np
from IPython.display import display

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.width", 1000)

In [9]:
data = pd.read_csv("/home/antonios/Desktop/Practica_de_vara/data-science-internship/data/raw/hotel_booking_cancellation_prediction.csv")
# data_missing = pd.read_csv("/home/antonios/Desktop/Practica_de_vara/data-science-internship/data/cleaned/01_missing_values_cleaned.csv")
# data_duplicate = pd.read_csv("/home/antonios/Desktop/Practica_de_vara/data-science-internship/data/cleaned/02_duplicates_cleaned.csv")
# data_statistical = pd.read_csv("/home/antonios/Desktop/Practica_de_vara/data-science-internship/data/cleaned/03_statistical_outliers_cleaned.csv")
# data_date_errors = pd.read_csv("/home/antonios/Desktop/Practica_de_vara/data-science-internship/data/cleaned/04_data_errors_cleaned.csv")
data_dtype = pd.read_pickle("/home/antonios/Desktop/Practica_de_vara/data-science-internship/data/cleaned/07_dtypes_cleaned.pkl")
final = pd.read_pickle("/home/antonios/Desktop/Practica_de_vara/data-science-internship/data/cleaned/final_cleaned_data.pkl")
encoded_data = pd.read_pickle("/home/antonios/Desktop/Practica_de_vara/data-science-internship/data/results/feature_engineered_data.pkl")
# load data 

Initial Data Exploration

In [19]:
print("=== Sample Data ===")
# display(data.head(10))
# display(data_missing.head(10))
# display(data_duplicate.head(10))
# display(data_statistical.head(10))
# display(final.head(10))
#display(encoded_data["high_risk_segment"])
display(final["distribution_channel"])

#display(final["deposit_type"])

print("\n=== Basic info ===")
# display(data_dtype.dtypes)

print("\n=== Data Shape ===")

#display(f"Rows: {data.shape[0]}, Columns: {data.shape[1]}")

=== Sample Data ===


0          Direct
1           TA/TO
2           TA/TO
3           TA/TO
4           TA/TO
5           TA/TO
6           TA/TO
7           TA/TO
8           TA/TO
9           TA/TO
10          TA/TO
11          TA/TO
12          TA/TO
13         Direct
14          TA/TO
15          TA/TO
16          TA/TO
17         Direct
18          TA/TO
19          TA/TO
20         Direct
21          TA/TO
22          TA/TO
23          TA/TO
24          TA/TO
25          TA/TO
26      Corporate
27          TA/TO
28          TA/TO
29          TA/TO
30          TA/TO
31          TA/TO
32          TA/TO
33          TA/TO
34          TA/TO
35          TA/TO
36          TA/TO
37          TA/TO
38          TA/TO
39          TA/TO
40          TA/TO
41          TA/TO
42          TA/TO
43      Corporate
44          TA/TO
45          TA/TO
46          TA/TO
47          TA/TO
48          TA/TO
49          TA/TO
50          TA/TO
51          TA/TO
52          TA/TO
53          TA/TO
54          TA/TO
55        


=== Basic info ===

=== Data Shape ===


In [ ]:
hotel_data = data.copy()
# copy dataset for manipulation

Summary of Numerical/Categorical columns

In [ ]:
print("\n=== Numerical Columns Summary===")
display(hotel_data.describe())

print("\n=== Categorical Columns Summary ===")
display(hotel_data.describe(include="object"))

Column names and data types

In [ ]:
print("\n=== Column names")
#display(hotel_data.columns.to_list())

print("\n=== Data types ===")
display(hotel_data.dtypes)

Missing Values Check

In [ ]:
print("\n=== Missing values ===")
display(hotel_data.isnull().sum())

print("\n=== Missing value(procents) ===")
display(hotel_data.isnull().sum()/ len(hotel_data) * 100)

Check duplicates

In [ ]:
print("\n=== Check for duplicates===")
display(hotel_data.duplicated().sum())

Target variable analysis

In [ ]:
print("\n=== Target variable analysis")
hotel_data["reserved_room_type"].value_counts()
hotel_data["assigned_room_type"].value_counts()

#rint(hotel_data["company"].value_counts().head(5))

Outliers in Numerical / Categorical columns

In [ ]:
print("\n=== Identify outliers in numerical columns")
numerical_cols = hotel_data.select_dtypes(include=["int64","float64"]).columns
print(numerical_cols)
for col in numerical_cols:
    Q1 = hotel_data[col].quantile(0.25)
    Q2 = hotel_data[col].quantile(0.75)
    IQR = Q2 - Q1
    outliers = hotel_data[(hotel_data[col] < Q1 - 1.5*IQR) | (hotel_data[col] > Q2 + 1.5*IQR)]
    print(f"{col}: {len(outliers)} outliers")
    print("+" * 50) 

print("\n=== Identify outliers in categorical columns")
categorical_columns = hotel_data.select_dtypes(include="object").columns
for col in categorical_columns:
    display(f"{col}: {hotel_data[col].nunique()} unique elements") 
    display(hotel_data[col].value_counts().head())
    print("-" * 50)


Data consistency checks

Non-numeric values in numeric columns

In [ ]:
print("---Non-numeric values in numeric columns---")
for col in numerical_cols:
    non_numeric = hotel_data[col].astype(str).str.contains('[^0-9,-]', regex=True).sum() 
    if non_numeric > 0:
        print(f"{col}: {non_numeric} non-numeric values found")
        examples = hotel_data[hotel_data[col].astype(str).str.contains('[^0-9.-]', regex=True)][col].head()
        print(f"Examples: {examples.to_list()}")


Logical Relationship Checks

In [ ]:
# Total guests > 0
zero_guests = hotel_data[(hotel_data["adults"] + hotel_data["children"] + hotel_data["babies"]) == 0]
print(f"Bookings with 0 total guests: {len(zero_guests)}")

# Stay duration > 0
zero_nights = hotel_data[(hotel_data['stays_in_weekend_nights'] + hotel_data['stays_in_week_nights']) == 0]
print(f"Bookings with 0 nights stay: {len(zero_nights)}")

# Lead time >=0
negative_lead = hotel_data[hotel_data["lead_time"] < 0]
print(f"Bookings with negative lead time: {len(negative_lead)}")


invalid_repeated = hotel_data[~hotel_data['is_repeated_guest'].isin([0, 1])]
print(f"Invalid is_repeated_guest values: {len(invalid_repeated)}")


negative_adults = hotel_data[hotel_data['adults'] < 0]
print(f"Bookings with negative adults: {len(negative_adults)}")


negative_children = hotel_data[hotel_data['children'] < 0]
print(f"Bookings with negative children: {len(negative_children)}")


negative_babies = hotel_data[hotel_data['babies'] < 0]
print(f"Bookings with negative babies: {len(negative_babies)}")

Impossible Values

In [ ]:
negative_adr = hotel_data[hotel_data['adr'] < 0]
print(f"Bookings with negative ADR: {len(negative_adr)}")


negative_prev_cancel = hotel_data[hotel_data['previous_cancellations'] < 0]
print(f"Bookings with negative previous_cancellations: {len(negative_prev_cancel)}")


negative_prev_bookings = hotel_data[hotel_data['previous_bookings_not_canceled'] < 0]
print(f"Bookings with negative previous_bookings_not_canceled: {len(negative_prev_bookings)}")


negative_changes = hotel_data[hotel_data['booking_changes'] < 0]
print(f"Bookings with negative booking_changes: {len(negative_changes)}")


negative_waiting = hotel_data[hotel_data['days_in_waiting_list'] < 0]
print(f"Bookings with negative days_in_waiting_list: {len(negative_waiting)}")


negative_parking = hotel_data[hotel_data['required_car_parking_spaces'] < 0]
print(f"Bookings with negative required_car_parking_spaces: {len(negative_parking)}")


negative_requests = hotel_data[hotel_data['total_of_special_requests'] < 0]
print(f"Bookings with negative total_of_special_requests: {len(negative_requests)}")